<a href="https://colab.research.google.com/github/salarMokhtariL/Facke-News-Detection/blob/main/Fake_News_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas torch sklearn transformers

In [ ]:
!pip install transformers

In [ ]:
import pandas as pd
import torch

from sklearn.model_selection import train_test_split

from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.metrics import accuracy_score, classification_report